In [8]:
# Import Libraries
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd
from ta.volatility import BollingerBands
from ta.momentum import RSIIndicator
from ta.others import daily_return,cumulative_return
from plotly.offline import init_notebook_mode
import cufflinks as cf
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import os
import requests
from bs4 import BeautifulSoup
import warnings
from tqdm.auto import tqdm

warnings.filterwarnings("ignore")
%matplotlib inline
init_notebook_mode(connected=True)
cf.go_offline()

In [9]:
# Check if folder input exists

# Define the folder name
folder_name = '../input'

# Check if the folder exists
if not os.path.exists(folder_name):
    # If it doesn't exist, create the folder
    os.makedirs(folder_name)
    print(f"'{folder_name}' folder created.")
else:
    print(f"'{folder_name}' folder already exists.")

'../input' folder already exists.


In [10]:
# Check if folder tickers exists

# Define the folder name
folder_name = '../tickers'

# Check if the folder exists
if not os.path.exists(folder_name):
    # If it doesn't exist, create the folder
    os.makedirs(folder_name)
    print(f"'{folder_name}' folder created.")
else:
    print(f"'{folder_name}' folder already exists.")

'../tickers' folder already exists.


In [11]:
# Define the folder name
folder_name = '../input'

# Get the data
index_list = []
for file_name in os.listdir(folder_name):
    if os.path.isfile(os.path.join(folder_name, file_name)):
        index_list.append(file_name)

In [12]:
# Get the tickers data

# Define the folder name
folder_name = '../tickers'

wiki = pd.DataFrame()
for file_name in os.listdir(folder_name):
    f = os.path.join(folder_name, file_name)
    tickers = pd.read_csv(f)
    wiki = pd.concat([wiki,tickers],ignore_index=True)

# Remove duplicates
wiki = wiki.drop_duplicates(subset='Ticker', keep='first')
wiki.head(50)

,Unnamed: 0,Name,Sector,Ticker
0,1,Agile Content,Electrónica y software,AGIL.MC
1,2,Catenon,Electrónica y software,COM.MC
2,3,EiDF,Energías renovables,EIDF.MC
3,4,Asturiana de Laminados (elZinc),"Mineral, metales y transformación",ELZ.MC
4,5,Endurance Motive,Fabricación y montaje de bienes de equipo,END.MC
5,6,FacePhi Biometria,Electrónica y software,FACE.MC
6,7,Gigas Hosting,Electrónica y software,GIGA.MC
7,9,Holaluz,Energías renovables,HLZ.MC
8,10,Izertis,Telecomunicaciones y otros,IZER.MC
9,11,Lleida.net,Telecomunicaciones y otros,LLN.MC


In [13]:
# Settings
year = '2025'

In [14]:
# Plotly
fig = make_subplots()

# Graph (1,1)
# Loop all stock files and get cummulative return for year
for stock in tqdm(index_list):
    f = os.path.join("../input", stock)
    df = pd.read_csv(f,index_col="Date",parse_dates=True)
    df['Cummulative_Return'] = cumulative_return(df["Adj Close"].loc[year])
    fig.add_trace(go.Scatter(x=df['Cummulative_Return'].loc[year].index,y=df['Cummulative_Return'].loc[year],mode="lines",name=stock.split('.')[0]),row=1,col=1)

# Update xaxis properties
fig.update_yaxes(title_text="Return", row=1, col=1)

fig.update_layout(height=800, width=1300,showlegend=True,title=f"Cummulative Returns for {year}")

fig.show()

  0%|          | 0/785 [00:00<?, ?it/s]

KeyError: '2025'

In [ ]:
# Create also table of cummulative returns
list = []

for stock in tqdm(index_list):
    f = os.path.join("../input",stock)
    df = pd.read_csv(f,index_col="Date",parse_dates=True)
    df['Cummulative_Return'] = cumulative_return(df["Adj Close"].loc[year])
    list.append([df.loc[year].tail(1).index.item(),stock.split('.csv')[0],df["Cummulative_Return"].loc[year].iloc[-1]])


cum = pd.DataFrame(list, columns=['Date','Ticker','Cummulative_Return'])
cum = wiki[['Ticker','Name','Sector']].merge(cum,on='Ticker',how='left')
cum = cum.sort_values(by=['Cummulative_Return'],ignore_index=True, ascending=False)
cum.head(40)



  0%|          | 0/781 [00:00<?, ?it/s]

,Ticker,Name,Sector,Date,Cummulative_Return
0,DIA.MC,DIA,Comercio,2025-03-20,119507.837475
1,TKA.DE,Thyssenkrupp AG,Conglomerate,2025-03-20,145.956974
2,RHM.DE,Rheinmetall,Aerospace & Defence,2025-03-20,120.447020
3,ORY.MC,Oryzon Genomics,Productos farmacéuticos y biotecnología,2025-03-20,113.049274
4,HAG.DE,Hensoldt AG,NaN,2025-03-20,101.064461
5,HO.PA,Thales,Industrials,2025-03-20,76.526151
6,PRS.MC,Prisa,Medios de comunicación y publicidad,2025-03-20,62.376240
7,EIDF.MC,EiDF,Energías renovables,2025-03-20,61.769351
8,CVS,CVS Health,Health Care,2025-03-20,56.669888
9,GLE.PA,Société Générale,Financial Services,2025-03-20,56.093060


In [ ]:
cum.tail(40)

,Ticker,Name,Sector,Date,Cummulative_Return
741,ALGN,Align Technology,Health Care,2025-03-20,-18.604012
742,HUBB,Hubbell Incorporated,Industrials,2025-03-20,-18.714644
743,PYPL,PayPal,Financials,2025-03-20,-18.879086
744,NTAP,NetApp,Information Technology,2025-03-20,-18.908145
745,LEG.DE,LEG Immobilien AG,Real estate,2025-03-20,-18.938834
746,LW,Lamb Weston,Consumer Staples,2025-03-20,-18.956209
747,DAY,Dayforce,Industrials,2025-03-20,-19.071296
748,AIXA.DE,Aixtron SE,Semiconductor industry,2025-03-20,-19.456556
749,SRE,Sempra,Utilities,2025-03-20,-19.614366
750,DAL,Delta Air Lines,Industrials,2025-03-20,-19.781279


In [ ]:
# Create also table of daily returns
list = []

for stock in tqdm(index_list):
    f = os.path.join("../input",stock)
    df = pd.read_csv(f,index_col="Date",parse_dates=True)
    df['Daily_Return'] = daily_return(df["Adj Close"].loc[year])
    list.append([df.loc[year].tail(1).index.item(),stock.split('.csv')[0],df["Daily_Return"].loc[year].iloc[-1]])


cum = pd.DataFrame(list, columns=['Date','Ticker','Daily_Return'])
cum = wiki[['Ticker','Name','Sector']].merge(cum,on='Ticker',how='left')
cum = cum.sort_values(by=['Daily_Return'],ignore_index=True, ascending=False)
cum.head(40)

  0%|          | 0/781 [00:00<?, ?it/s]

,Ticker,Name,Sector,Date,Daily_Return
0,XRP-EUR,Ripple,Cryptocurrency,2025-03-20,7.682459
1,ERF.PA,Eurofins Scientific,Healthcare,2025-03-20,6.734692
2,JBL,Jabil,Information Technology,2025-03-20,5.205054
3,DRI,Darden Restaurants,Consumer Discretionary,2025-03-20,5.079729
4,ADA-EUR,Cardano,Cryptocurrency,2025-03-20,4.271699
5,APP,AppLovin,Information Technology,2025-03-20,3.848094
6,SOL-EUR,Solana,Cryptocurrency,2025-03-20,3.804971
7,LUV,Southwest Airlines,Industrials,2025-03-19,3.735718
8,ALL,Allstate,Financials,2025-03-20,3.355787
9,META,Meta Platforms,Communication Services,2025-03-20,3.114406
